# VacationPy
----



#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = '../output_data/cities.csv'

weather_df = pd.read_csv(csv_file)
weather_df = weather_df.drop(columns=['Unnamed: 0'])
weather_df.head()


,city,lat,lng,max_temp,humidity,cloudiness,windspeed
0,Alta Floresta,-9.8756,-56.0861,21.52,99,100,1.22
1,Mehamn,71.0357,27.8492,-1.98,84,100,8.13
2,Jamestown,42.0970,-79.2353,4.73,99,90,6.26
3,Lorengau,-2.0226,147.2712,29.01,76,69,3.20
4,Mogadishu,2.0371,45.3438,26.88,76,40,6.34


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = weather_df[["lat","lng"]].astype(float)
humidity = weather_df["humidity"].astype(float)

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
Figure(layout=FigureLayout(height='420px'))

NameError: name 'Figure' is not defined

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
fitting_weather_df = weather_df.loc[(weather_df["windspeed"] <= 10) & (weather_df["cloudiness"]== 0) & \
                  (weather_df["max_temp"] >= 20) & (weather_df["max_temp"] <= 28)].dropna()
fitting_weather_df


,city,lat,lng,max_temp,humidity,cloudiness,windspeed
32,Cayenne,4.9333,-52.3333,26.72,87,0,1.34
38,Byron Bay,-28.6500,153.6167,27.97,35,0,2.06
78,Atuona,-9.8000,-139.0333,25.48,70,0,5.41
123,Khāga,25.7833,81.1167,22.74,46,0,1.56
164,Salalah,17.0151,54.0924,24.05,53,0,1.54
174,Luxor,25.6989,32.6421,20.18,83,0,1.59
195,Saint-Pierre,-21.3393,55.4781,27.80,65,0,2.57
202,Jiwani,25.0500,61.7417,24.76,46,0,3.64
220,Budaun,28.0500,79.1167,21.46,38,0,0.82
289,Yulara,-25.2406,130.9889,24.91,18,0,2.57


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = fitting_weather_df.loc[:,["city","lat","lng"]]

hotel_df["Hotel Name"] = ""
hotel_df.head()

,city,lat,lng,Hotel Name
32,Cayenne,4.9333,-52.3333,
38,Byron Bay,-28.6500,153.6167,
78,Atuona,-9.8000,-139.0333,
123,Khāga,25.7833,81.1167,
164,Salalah,17.0151,54.0924,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}
for index, row in hotel_df.iterrows():
    lat = row["lat"]
    lng = row["lng"]
    city_name = row["city"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 32: Cayenne.
Closest hotel in Cayenne is Grand Hotel Montabo.
--------------------Ending Search
Retrieving Results for Index 38: Byron Bay.
Closest hotel in Byron Bay is The Park Hotel Motel.
--------------------Ending Search
Retrieving Results for Index 78: Atuona.
Closest hotel in Atuona is Hôtel Hiva Oa Hanakee Lodge.
--------------------Ending Search
Retrieving Results for Index 123: Khāga.
Closest hotel in Khāga is Maurya naashta Bhandar.
--------------------Ending Search
Retrieving Results for Index 164: Salalah.
Closest hotel in Salalah is Salalah Gardens Hotel.
--------------------Ending Search
Retrieving Results for Index 174: Luxor.
Closest hotel in Luxor is Hilton Luxor Resort & Spa.
--------------------Ending Search
Retrieving Results for Index 195: Saint-Pierre.
Closest hotel in Saint-Pierre is Hôtel Le Saint-Pierre.
--------------------Ending Search
Retrieving Results for Index 202: Jiwani.
Closest hotel in Jiwani is Hamza hotel jiwani.
------

In [9]:
hotel_df.head(10)

,city,lat,lng,Hotel Name
32,Cayenne,4.9333,-52.3333,Grand Hotel Montabo
38,Byron Bay,-28.6500,153.6167,The Park Hotel Motel
78,Atuona,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
123,Khāga,25.7833,81.1167,Maurya naashta Bhandar
164,Salalah,17.0151,54.0924,Salalah Gardens Hotel
174,Luxor,25.6989,32.6421,Hilton Luxor Resort & Spa
195,Saint-Pierre,-21.3393,55.4781,Hôtel Le Saint-Pierre
202,Jiwani,25.0500,61.7417,Hamza hotel jiwani
220,Budaun,28.0500,79.1167,Hotel J.P. Palace
289,Yulara,-25.2406,130.9889,Sails in the Desert


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
Figure(layout=FigureLayout(height='420px'))

NameError: name 'Figure' is not defined